TODO
* ~Merge Tables 2,4,10 to work together~
* ~Create loop for full season~
* ~Add season and episode to front of data frame~

Someday:
* Fix Italics:
    * Work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
    * Not work: 
        * italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
        
Lessons Learned: 
* Look for specific class types in BeautifulSoup. The fact that the transcript was formated as a *wikitable* made life much easier
* Figuring out Edge Cases can take a majority of work

In [1]:
# imports

#standard
import pandas as pd
#import random
from datetime import datetime
from pandasql import sqldf
pysqldf = lambda q: sqldf(q,globals())

#Viz
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

#Data Pulls
from bs4 import BeautifulSoup
import requests
import re

In [2]:
#Base URL for Transcripts
url_base = 'https://avatar.fandom.com/wiki/Avatar_Wiki:Transcripts'
response = requests.get(url_base)

#Count # of Tables of Transcript Page
soup = BeautifulSoup(response.content, "html.parser")
tables = soup.find_all("table")
print(f"Found {len(tables)} tables on the page.")

# Table 2: Book 1 Water
# Table 4: Book 2 Earth
# Table 10: Book 3 Fire

# if len(tables) > 0:
#     print("First table:")
#     print(tables[10].prettify())

Found 90 tables on the page.


In [3]:
book_list = [2, #Water
             5, #Earth
             10] # Fire

episode_list = []

for book in book_list:
    specific_table = soup.find_all("table")[book]  # Replace 0 with the index of the table you want
    links = specific_table.find_all("a")
    for link in links:
        episode_list.append(link.get("href"))

In [4]:
#Remove url links for commentary links
episode_list_filtered = [episode for episode in episode_list if "commentary" not in episode]
#episode_list_filtered

#Creates Episodes URLs
base = 'https://avatar.fandom.com'
url_episode_list = []
for episode in episode_list_filtered:
    #print(base + episode)
    url_episode_list.append(base + episode)

### Edge Case Identifier

In [ ]:
#Printe Edge Case Episodes
for episode_number, url in enumerate(url_episode_list):
    
    split_text = url.split(':', 2)
    episode_title = split_text[2]

    episode_number += 1
    # print(episode_number)
    # print(episode_title)
    # print(url)
    
    response = requests.get(url)
    # Parse the page with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all the tables with the class "wikitable"
    wikitable_tables = soup.find_all("table", class_="wikitable")
    num_tables = len(wikitable_tables)
    
    if num_tables >= 2:
            print(episode_number)
            print(episode_title)
            print(url)
            print(f"Number of Tables: {len(wikitable_tables)}")

In [ ]:
# Edge Case Exploration

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Boy_in_the_Iceberg'
# Index 1 | 2nd Table

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Avatar_Returns'
# Index 1 2nd Table 

# url = 'https://avatar.fandom.com/wiki/Transcript:Bato_of_the_Water_Tribe'
# Index 0 1st Table 
# 2nd Table contains a deleted scene

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko

#url = 'https://avatar.fandom.com/wiki/Transcript:The_Puppetmaster'
# Index 1 2nd Table 
# Removes the "previously on Avatar"
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

data = []

print(f"Number of Tables: {len(tables)}")
print()

desired_table = tables[:4]  
# Find all rows within the 2nd Table that holds the transcript
rows = desired_table.select("tbody tr")

for row in rows:
    character_cell = row.find('th')  # Find the <th> for character names
    dialogue_cell = row.find('td')    # Find the <td> for character's dialogue

    if character_cell and dialogue_cell:  # Ensure both character & dialogue are present (thus skipping pure screen directions)
        character = character_cell.get_text(strip=True)

        # Replace <i> tags with their text (ITALICS CAUSING PROBLEMS)
        for italics in dialogue_cell.find_all('i'):
            # Get the text inside <i>, append '123' and replace the <i> with this text
            italic_text_with_suffix =  '12345' + italics.get_text(strip=True) + '12345'
            italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

        dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics

        data.append({'character': character, 'dialogue': dialogue})

        df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

df

In [ ]:
#Tales of Ba Sing Se - Edge Case Exploration
url = 'https://avatar.fandom.com/wiki/Transcript:The_Tales_of_Ba_Sing_Se'
# Index 0 The Tale of Toph and Katara
# Index 1 The Tale of Iroh
# Index 2 The Tale of Aang
# Index 3 The Tale of Sokka 
# Index 4 THe Tale of Zuko
    
response = requests.get(url)
ep_soup = BeautifulSoup(response.content, "html.parser")
tables = ep_soup.select("table.wikitable")

data = []

# Iterate over the first 5 tables (0 to 4)
for table in tables[:5]:  
    # Find all rows within each table
    rows = table.select("tbody tr")

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Process the <i> tags within the dialogue
            for italics in dialogue_cell.find_all('i'):
                # Prepend and append '12345' to italicized text
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            # Get the full dialogue text after modifications
            dialogue = dialogue_cell.get_text(strip=True)

            # Append the data to the list
            data.append({'character': character, 'dialogue': dialogue})

# Convert the collected data into a DataFrame
df = pd.DataFrame(data)

#Cleaning
df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()
df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()

#Word Count
df['word_count'] = df['dialogue'].str.split().apply(len)

pd.set_option('display.max_rows', None)

df

### Full Series Trancript Pull
* Includes all Non Edge Cases
* All edge cases including:
    * Where primary table is index 1 (rather than index 0) 
    * Tales of Ba Sing Se

In [5]:
#episodes, so technically need to subtract 1 from each to get the edge case
edge_case_episodes = [1, #The_Boy_in_the_Iceberg
                      2, #The_Avatar_Returns
                      #15, #Bato_of_the_Water_Tribe - Use table 1, index 0 to remove deleted scence
                      35, #The_Tales_of_Ba_Sing_Se
                      48 #The_Puppetmaster
                     ]

#Subtract 1 from each episode to convert episode_number --> index
edge_case_episodes = [episode - 1 for episode in edge_case_episodes]
#print(edge_case_episodes)

In [6]:
transcript_df = pd.DataFrame()

for episode_number, url in enumerate(url_episode_list):
    # Splits based on Edge Cases
    
    data = []  # Initialize data list for an individual episode
    
    # If Statement splits data into base and non-edge cases
    if episode_number not in edge_case_episodes:
        
        response = requests.get(url)
        ep_soup = BeautifulSoup(response.content, "html.parser")
        tables = ep_soup.select("table.wikitable")
        
        desired_table = tables[0]
        rows = desired_table.select("tbody tr")

    if episode_number in edge_case_episodes:
            
            if episode_number != 34:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                desired_table = tables[1]
                rows = desired_table.select("tbody tr")
                
            else:
                response = requests.get(url)
                ep_soup = BeautifulSoup(response.content, "html.parser")
                tables = ep_soup.select("table.wikitable")
                
                all_rows = []  # Initialize list to store all rows from all tables

                for table in tables[:5]:
                    table_rows = table.select("tbody tr")
                    all_rows.extend(table_rows)  # Append all rows to the all_rows list
                    
                rows = all_rows  # Now, 'rows' contains all the rows from tables 0 to 4
                    
    split_text = url.split(':', 2)
    episode_title = split_text[2]
    episode_number += 1
    
    # print(episode_number)
    # print(episode_title)
    # print(url)
    # print()

    for row in rows:
        character_cell = row.find('th')  # Find the <th> for character names
        dialogue_cell = row.find('td')   # Find the <td> for character's dialogue

        if character_cell and dialogue_cell:  # Ensure both character & dialogue are present
            character = character_cell.get_text(strip=True)

            # Replace <i> tags with their text (handle italicized text)
            for italics in dialogue_cell.find_all('i'):
                italic_text_with_suffix = '12345' + italics.get_text(strip=True) + '12345'
                italics.replace_with(italic_text_with_suffix)  # Replace <i> tag with modified text

            dialogue = dialogue_cell.get_text(strip=True)  # Get the text after modifying italics
            data.append({'character': character, 'dialogue': dialogue})

    # Create DataFrame if the data was extracted
    if data:
        df = pd.DataFrame(data)

        # Cleaning the dialogue column
        df['dialogue'] = df['dialogue'].str.replace(r'\[.*?\]', '', regex=True).str.strip()  # Remove text in []
        df['dialogue'] = df['dialogue'].str.replace(r'\(.*?\)', '', regex=True).str.strip()  # Remove text in ()
        df['dialogue'] = df['dialogue'].str.replace('12345', ' ').str.strip()  # Replace '12345' with space

        # Add word count
        df['word_count'] = df['dialogue'].str.split().apply(len)

        # Add episode number and title
        df['abs_episode'] = episode_number
        df['episode_title'] = episode_title 

        # Concatenate with the main transcript DataFrame
        transcript_df = pd.concat([transcript_df, df], ignore_index=True)

### Edge Case Check 

In [ ]:
transcript_df[transcript_df['episode'] ==2].head(126)

In [ ]:
transcript_df[transcript_df['episode'] ==35].head(126)

In [ ]:
transcript_df[transcript_df['episode'] ==48].tail()

### Transcript Finalization

In [ ]:
#transcript_df

In [7]:
from urllib.parse import unquote
#Replace apostrophes 
transcript_df['episode_title'] = transcript_df['episode_title'].apply(lambda x: unquote(x))

#Replace '_(episode)'
transcript_df['episode_title'] = transcript_df['episode_title'].str.replace('_(episode)', '', regex=False)

In [8]:
episode_season_mapping = {
    'The_Boy_in_the_Iceberg': (1, 1),
    'The_Avatar_Returns': (1, 2),
    'The_Southern_Air_Temple': (1, 3),
    'The_Warriors_of_Kyoshi': (1, 4),
    'The_King_of_Omashu': (1, 5),
    'Imprisoned': (1, 6),
    'Winter_Solstice,_Part_1:_The_Spirit_World': (1, 7),
    'Winter_Solstice,_Part_2:_Avatar_Roku': (1, 8),
    'The_Waterbending_Scroll': (1, 9),
    'Jet': (1, 10),
    'The_Great_Divide': (1, 11),
    'The_Storm': (1, 12),
    'The_Blue_Spirit': (1, 13),
    'The_Fortuneteller': (1, 14),
    'Bato_of_the_Water_Tribe': (1, 15),
    'The_Deserter': (1, 16),
    'The_Northern_Air_Temple': (1, 17),
    'The_Waterbending_Master': (1, 18),
    'The_Siege_of_the_North,_Part_1': (1, 19),
    'The_Siege_of_the_North,_Part_2': (1, 20),
    'The_Avatar_State': (2, 1),
    'The_Cave_of_Two_Lovers': (2, 2),
    'Return_to_Omashu': (2, 3),
    'The_Swamp': (2, 4),
    'Avatar_Day': (2, 5),
    'The_Blind_Bandit': (2, 6),
    'Zuko_Alone': (2, 7),
    'The_Chase': (2, 8),
    'Bitter_Work': (2, 9),
    'The_Library': (2, 10),
    'The_Desert': (2, 11),
    "The_Serpent's_Pass": (2, 12),
    'The_Drill': (2, 13),
    'City_of_Walls_and_Secrets': (2, 14),
    'The_Tales_of_Ba_Sing_Se': (2, 15),
    "Appa's_Lost_Days": (2, 16),
    'Lake_Laogai': (2, 17),
    'The_Earth_King': (2, 18),
    'The_Guru': (2, 19),
    'The_Crossroads_of_Destiny': (2, 20),
    'The_Awakening': (3, 1),
    'The_Headband': (3, 2),
    'The_Painted_Lady': (3, 3),
    "Sokka's_Master": (3, 4),
    'The_Beach': (3, 5),
    'The_Avatar_and_the_Fire_Lord': (3, 6),
    'The_Runaway': (3, 7),
    'The_Puppetmaster': (3, 8),
    'Nightmares_and_Daydreams': (3, 9),
    'The_Day_of_Black_Sun,_Part_1:_The_Invasion': (3, 10),
    'The_Day_of_Black_Sun,_Part_2:_The_Eclipse': (3, 11),
    'The_Western_Air_Temple': (3, 12),
    'The_Firebending_Masters': (3, 13),
    'The_Boiling_Rock,_Part_1': (3, 14),
    'The_Boiling_Rock,_Part_2': (3, 15),
    'The_Southern_Raiders': (3, 16),
    'The_Ember_Island_Players': (3, 17),
    "Sozin's_Comet,_Part_1:_The_Phoenix_King": (3, 18),
    "Sozin's_Comet,_Part_2:_The_Old_Masters": (3, 19),
    "Sozin's_Comet,_Part_3:_Into_the_Inferno": (3, 20),
    "Sozin's_Comet,_Part_4:_Avatar_Aang": (3, 21),
}

In [9]:
transcript_df['season'] = transcript_df['episode_title'].map(lambda x: episode_season_mapping.get(x, (None, None))[0])
transcript_df['episode_number'] = transcript_df['episode_title'].map(lambda x: episode_season_mapping.get(x, (None, None))[1])

In [10]:
transcript_df.sample(10).sort_index()

,character,dialogue,word_count,abs_episode,episode_title,season,episode_number
178,Sokka,"Let's see your bison fly now, air boy.",8,2,The_Avatar_Returns,1,2
994,Warden,Show no mercy!,3,6,Imprisoned,1,6
1247,Shyu,"A few weeks ago, an amazing thing occurred. Th...",18,8,"Winter_Solstice,_Part_2:_Avatar_Roku",1,8
1849,Gan Jin leader,See! We're going to become part of the food ch...,13,11,The_Great_Divide,1,11
2201,Zhao,"Uhh, no. Tell me, how does it feel to be the o...",82,13,The_Blue_Spirit,1,13
3525,Fong,"So, it's decided then. I'll help you figure ou...",22,21,The_Avatar_State,2,1
4120,Due,You want me to eat old Slim? He's like a membe...,14,24,The_Swamp,2,4
7658,Toph,Of course you do. And that's why this wanted ...,17,47,The_Runaway,3,7
9229,Katara,"But we were too late. When we got there, the m...",17,56,The_Southern_Raiders,3,16
9719,Bumi,Wait! Someone's missing from your group. Some...,11,59,"Sozin's_Comet,_Part_2:_The_Old_Masters",3,19


In [11]:
transcript_df.to_csv('full_transcript.csv')